In [132]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass
import random
from scipy.optimize import differential_evolution
from tqdm.notebook import tqdm
from datetime import datetime, timedelta, timezone

@dataclass(frozen=True)  # Make the dataclass immutable
class LiquidityZone:
    start_idx: pd.Timestamp
    green_end_idx: pd.Timestamp
    end_idx: pd.Timestamp
    high: float
    low: float
    entry_price: float
    height_pct: float
    
    def __hash__(self):
        return hash((self.start_idx, self.green_end_idx, self.end_idx))
    
    def __eq__(self, other):
        if not isinstance(other, LiquidityZone):
            return False
        return (self.start_idx == other.start_idx and 
                self.green_end_idx == other.green_end_idx and 
                self.end_idx == other.end_idx)

class TradingStrategy:
    LZ_MIN_CANDLES = 3
    
    def __init__(self, 
                 symbol: str = None, 
                 data: pd.DataFrame = None,
                 start_date: Optional[datetime] = None,
                 end_date: Optional[datetime] = None,
                 days: Optional[int] = None,
                 interval: str = '1h',
                 rsi_period: int = 14,
                 rsi_smooth_period: int = 10,
                 lz_width: int = 20,
                 exit_threshold_pct: float = 1.0,
                 entry_height_pct: float = 1.0,
                 min_zone_height_pct: float = 0.5,
                 atr_period: int = 10,
                 atr_factor: float = 3,
                 pivot_period: int = 2,
                 lower_smooth: int = 50,  
                 upper_smooth: int = 80,
                 alma_period: int = 9,
                 alma_offset: float = 0.85,
                 alma_sigma: float = 6): 
        
        # Store parameters
        self.params = {
            'rsi_period': rsi_period,
            'rsi_smooth_period': rsi_smooth_period,
            'lz_width': lz_width,
            'exit_threshold_pct': exit_threshold_pct,
            'entry_height_pct': entry_height_pct,
            'min_zone_height_pct': min_zone_height_pct,
            'atr_period': atr_period,
            'atr_factor': atr_factor,
            'pivot_period': pivot_period,
            'lower_smooth': lower_smooth,
            'upper_smooth': upper_smooth,
            'alma_period': alma_period,
            'alma_offset': alma_offset,
            'alma_sigma': alma_sigma
        }
        
        # Initialize data
        if data is not None:
            self.data = data.copy()
        else:
            if symbol is None:
                raise ValueError("Either 'data' or 'symbol' must be provided")
                
            # Handle date range
            end_date = pd.to_datetime(end_date) if end_date else datetime.now()
            if start_date:
                start_date = pd.to_datetime(start_date)
            elif days:
                start_date = end_date - timedelta(days=days)
            else:
                start_date = end_date - timedelta(days=365)
                
            # Download data
            self.data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
            
        # Validate data
        required_columns = ['Open', 'High', 'Low', 'Close']
        if not all(col in self.data.columns for col in required_columns):
            raise ValueError(f"Data must contain columns: {required_columns}")
        
        # Pre-calculate commonly used values
        self.data['returns'] = self.data['Close'].pct_change()
        self.data['high_low'] = self.data['High'] - self.data['Low']
        self.data['candle_type'] = (self.data['Close'] > self.data['Open']).astype(int)
        
        # Initialize containers
        self.signals = pd.DataFrame(index=self.data.index, columns=['Position'], data=0)
        self.liquidity_zones = []
        self.currently_holding = False
        self.active_trade_zone = None
        
        # Prepare indicators using vectorized operations
        self.prepare_indicators()

    def calculate_alma(self, data: pd.Series) -> pd.Series:
        """Vectorized ALMA calculation"""
        period = self.params['alma_period']
        offset = self.params['alma_offset']
        sigma = self.params['alma_sigma']
        
        m = offset * (period - 1)
        s = period / sigma
        
        # Vectorized weight calculation
        x = np.arange(period)
        weights = np.exp(-((x - m) ** 2) / (2 * s * s))
        weights = weights / weights.sum()
        
        # Use numpy's convolve for efficient calculation
        alma = pd.Series(
            np.convolve(data, weights, mode='valid'),
            index=data.index[period-1:]
        )
        
        return pd.concat([pd.Series([np.nan] * (period-1), index=data.index[:period-1]), alma])

    def prepare_indicators(self) -> None:
        """Vectorized indicator calculations"""
        # ALMA calculation
        self.data['ALMA'] = self.calculate_alma(self.data['Close'])
        self.data['ALMA_Slope'] = self.data['ALMA'].pct_change(3) * 100

        # Vectorized RSI calculation
        delta = self.data['Close'].diff()
        gain = (delta.where(delta > 0, 0)
               .rolling(window=self.params['rsi_period'])
               .mean())
        loss = (-delta.where(delta < 0, 0)
               .rolling(window=self.params['rsi_period'])
               .mean())
        
        rs = gain / loss
        self.data['RSI'] = 100 - (100 / (1 + rs))
        self.data['RSI_SMA'] = self.data['RSI'].rolling(
            window=self.params['rsi_smooth_period']
        ).mean()

        # Vectorized ATR calculation
        tr = pd.concat([
            self.data['high_low'],
            abs(self.data['High'] - self.data['Close'].shift()),
            abs(self.data['Low'] - self.data['Close'].shift())
        ], axis=1).max(axis=1)
        
        self.data['ATR'] = tr.rolling(window=self.params['atr_period']).mean()
        
        # Vectorized center line calculation
        self.calculate_center_line()
        
        # ATR Bands
        self.data['Upper_Band'] = (
            self.data['Center'] + 
            (self.params['atr_factor'] * self.data['ATR'])
        )
        self.data['Lower_Band'] = (
            self.data['Center'] - 
            (self.params['atr_factor'] * self.data['ATR'])
        )

    def calculate_center_line(self) -> None:
        """Vectorized center line calculation"""
        period = self.params['pivot_period']
        
        # Calculate rolling high and low
        rolling_high = self.data['High'].rolling(window=period).max()
        rolling_low = self.data['Low'].rolling(window=period).min()
        
        # Calculate center points
        center_points = (rolling_high + rolling_low) / 2
        
        # Smoothing with exponential weighted average
        self.data['Center'] = center_points.ewm(span=period, adjust=False).mean()

    def identify_liquidity_zones(self) -> None:
        """
        Identify liquidity zones based on consecutive green candles.
        A valid zone requires:
        1. Minimum 3 consecutive green candles
        2. Zone low = First green candle's high
        3. Zone high = Last green candle's low
        4. Zone height must meet minimum percentage requirement
        """
        self.liquidity_zones = []

        i = 0
        while i < len(self.data) - self.LZ_MIN_CANDLES:
            # Find start of green candle sequence
            if self.data['Close'].iloc[i] <= self.data['Open'].iloc[i]:  # Not a green candle
                i += 1
                continue

            # Count consecutive green candles
            green_count = 0
            start_idx = i

            while (i < len(self.data) and 
                   self.data['Close'].iloc[i] > self.data['Open'].iloc[i]):
                green_count += 1
                i += 1

            # Check if we have minimum required green candles
            if green_count >= self.LZ_MIN_CANDLES:
                # Zone boundaries
                zone_low = self.data['High'].iloc[start_idx]  # First green candle's high
                zone_high = self.data['Low'].iloc[i-1]  # Last green candle's low
                end_idx = min(i + self.params['lz_width'], len(self.data)-1)

                # Calculate zone metrics
                avg_price = (zone_high + zone_low) / 2
                zone_height_pct = ((zone_high - zone_low) / avg_price) * 100

                # Check if zone meets minimum height requirement
                if zone_height_pct >= self.params['min_zone_height_pct']:
                    entry_price = zone_low + (zone_high - zone_low) * self.params['entry_height_pct']

                    self.liquidity_zones.append(LiquidityZone(
                        start_idx=self.data.index[start_idx],
                        green_end_idx=self.data.index[i-1],
                        end_idx=self.data.index[end_idx],
                        high=zone_high,
                        low=zone_low,
                        entry_price=entry_price,
                        height_pct=zone_height_pct
                    ))
            else:
                i += 1  # Move to next candle if sequence wasn't long enough

        # Sort zones by time
        self.liquidity_zones.sort(key=lambda x: x.start_idx)

    def generate_signals(self) -> None:
        """Generate trading signals with forward-looking bias removed"""
        self.signals['Position'] = 0
        self.currently_holding = False
        self.active_trade_zone = None
        used_zones_starts = set()

        # Track potential liquidity zones as they form
        current_green_sequence = 0
        potential_zone_start = None
        confirmed_zones = []

        for i in range(1, len(self.data)):
            current_price = self.data['Close'].iloc[i]
            current_time = self.data.index[i]

            # Update green candle sequence
            if self.data['Close'].iloc[i] > self.data['Open'].iloc[i]:
                if current_green_sequence == 0:
                    potential_zone_start = i
                current_green_sequence += 1
            else:
                # When green sequence ends, confirm zone if valid
                if current_green_sequence >= self.LZ_MIN_CANDLES:
                    zone_low = self.data['High'].iloc[potential_zone_start]
                    zone_high = self.data['Low'].iloc[i-1]
                    end_idx = min(i + self.params['lz_width'], len(self.data)-1)

                    avg_price = (zone_high + zone_low) / 2
                    zone_height_pct = ((zone_high - zone_low) / avg_price) * 100

                    if zone_height_pct >= self.params['min_zone_height_pct']:
                        entry_price = zone_low + (zone_high - zone_low) * self.params['entry_height_pct']
                        confirmed_zones.append(LiquidityZone(
                            start_idx=self.data.index[potential_zone_start],
                            green_end_idx=self.data.index[i-1],
                            end_idx=self.data.index[end_idx],
                            high=zone_high,
                            low=zone_low,
                            entry_price=entry_price,
                            height_pct=zone_height_pct
                        ))
                current_green_sequence = 0
                potential_zone_start = None

            # Position management
            if self.currently_holding:
                if self.active_trade_zone:
                    exit_price = self.active_trade_zone.low * (1 - self.params['exit_threshold_pct']/100)
                    price_exit = current_price < exit_price
                    atr_exit = current_price < self.data['Lower_Band'].iloc[i]

                    if price_exit or atr_exit:
                        self.signals.iloc[i] = 0
                        self.currently_holding = False
                        used_zones_starts.add(self.active_trade_zone.start_idx)
                        self.active_trade_zone = None
                    else:
                        self.signals.iloc[i] = 1
            else:
                # Entry conditions using only confirmed zones
                alma_trend = current_price > self.data['ALMA'].iloc[i]
                rsi_condition = (self.params['lower_smooth'] <= 
                               self.data['RSI_SMA'].iloc[i] <= 
                               self.params['upper_smooth'])

                valid_zone = None
                for zone in confirmed_zones:
                    if zone.start_idx in used_zones_starts:
                        continue

                    if zone.start_idx <= current_time <= zone.end_idx:
                        zone_height = zone.high - zone.low
                        entry_top = zone.low + (zone_height * self.params['entry_height_pct'])

                        if zone.low <= current_price <= entry_top:
                            valid_zone = zone
                            break

                if alma_trend and rsi_condition and valid_zone:
                    self.signals.iloc[i] = 1
                    self.currently_holding = True
                    self.active_trade_zone = valid_zone

    def plot_strategy(self, days_to_plot=30):
        """Plot the strategy signals with forward-looking bias removed"""
        end_date = self.data.index[-1]
        start_date = end_date - timedelta(days=days_to_plot)

        plot_data = self.data[self.data.index >= start_date]
        plot_signals = self.signals[self.signals.index >= start_date]

        fig, ax = plt.subplots(figsize=(15, 8))

        # Plot candlesticks
        for i in range(len(plot_data)):
            date = plot_data.index[i]
            open_price = plot_data['Open'].iloc[i]
            close = plot_data['Close'].iloc[i]
            high = plot_data['High'].iloc[i]
            low = plot_data['Low'].iloc[i]

            color = 'g' if close > open_price else 'r'
            ax.vlines(date, low, high, color=color)
            ax.vlines(date, min(open_price, close), max(open_price, close), 
                     color=color, linewidth=4)

        # Plot ATR Bands and ALMA
        ax.plot(plot_data.index, plot_data['Upper_Band'], 
                label='Upper Band', color='purple', linewidth=1, linestyle='--')
        ax.plot(plot_data.index, plot_data['Lower_Band'], 
                label='Lower Band', color='purple', linewidth=1, linestyle='--')
        ax.plot(plot_data.index, plot_data['ALMA'], 
            label='ALMA', color='yellow', linewidth=2)

        # Plot liquidity zones with delayed visualization
        for zone in self.liquidity_zones:
            if zone.start_idx >= start_date and zone.start_idx <= end_date:
                # Shift zone visualization one candle after the green sequence
                shifted_start = zone.green_end_idx + pd.Timedelta(minutes=60)  # Assuming 1h timeframe

                rect = plt.Rectangle((shifted_start, zone.low), 
                                  zone.end_idx - shifted_start,
                                  zone.high - zone.low,
                                  facecolor='green', alpha=0.1)
                ax.add_patch(rect)

                ax.hlines(y=zone.entry_price,
                         xmin=shifted_start,
                         xmax=zone.end_idx,
                         colors='green',
                         linestyles='--',
                         alpha=0.8)

                # Plot exit level
                exit_price = zone.low * (1 - self.params['exit_threshold_pct']/100)
                ax.hlines(y=exit_price,
                         xmin=shifted_start,
                         xmax=zone.end_idx,
                         colors='red',
                         linestyles='--',
                         alpha=0.8)

        # Plot buy/sell signals
        buy_signals = plot_signals[plot_signals['Position'].diff() == 1]
        sell_signals = plot_signals[plot_signals['Position'].diff() == -1]

        ax.scatter(buy_signals.index, 
                  plot_data.loc[buy_signals.index, 'Low'] * 0.9,
                  marker='^', color='g', s=100, label='Buy')
        ax.scatter(sell_signals.index,
                  plot_data.loc[sell_signals.index, 'High'] * 1.1,
                  marker='v', color='r', s=100, label='Sell')

        # Formatting
        ax.grid(True, alpha=0.3)
        ax.set_title('Trading Strategy Signals')
        ax.set_xlabel('Date')
        ax.set_ylabel('Price')
        ax.legend()

        plt.xticks(rotation=45)
        plt.tight_layout()
        return fig
        
    def get_signals(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Get signals and data"""
        self.identify_liquidity_zones()
        self.generate_signals()
        return self.signals, self.data
    
    
        


In [133]:
class Backtester:
    def __init__(self, strategy, initial_capital=10000, trading_cost_pct=0.002, include_buy_hold=True):
        self.strategy = strategy
        self.initial_capital = initial_capital
        self.trading_cost_pct = trading_cost_pct
        self.include_buy_hold = include_buy_hold
        self.signals, self.data = strategy.get_signals()
        self.results = self._run_backtest()
        
    def _run_backtest(self):
        results = pd.DataFrame(index=self.data.index)
        results['Position'] = self.signals['Position']
        results['Price'] = self.data['Close']
        results['Signal'] = results['Position'].diff()

        # Calculate strategy returns
        results['Returns'] = results['Price'].pct_change()
        results['Strategy'] = results['Position'].shift(1) * results['Returns']
        
        # Buy & Hold returns
        if self.include_buy_hold:
            results['Buy_Hold'] = results['Returns']
            results['Buy_Hold_Equity'] = self.initial_capital * (1 + results['Buy_Hold']).cumprod()

        # Trading costs
        portfolio_value = self.initial_capital * (1 + results['Strategy']).cumprod()
        results['Trading_Cost'] = abs(results['Signal']) * self.trading_cost_pct * portfolio_value
        results['Strategy_Net'] = results['Strategy'] - (results['Trading_Cost'] / portfolio_value)

        results['Strategy_Equity'] = self.initial_capital * (1 + results['Strategy']).cumprod()
        results['Strategy_Net_Equity'] = self.initial_capital * (1 + results['Strategy_Net']).cumprod()

        # Calculate trade returns
        self.trade_returns = []
        entry_price = None
        entry_date = None
        entry_portfolio = None

        for idx, row in results.iterrows():
            if row['Signal'] == 1:
                entry_price = row['Price']
                entry_date = idx
                entry_portfolio = results.loc[idx, 'Strategy_Equity']
            elif row['Signal'] == -1 and entry_price is not None:
                exit_price = row['Price']
                exit_portfolio = results.loc[idx, 'Strategy_Equity']
                
                # Calculate returns with costs
                gross_return = (exit_price - entry_price) / entry_price
                entry_cost = entry_portfolio * self.trading_cost_pct
                exit_cost = exit_portfolio * self.trading_cost_pct
                total_cost_pct = (entry_cost + exit_cost) / entry_portfolio
                net_return = gross_return - total_cost_pct

                self.trade_returns.append({
                    'entry_date': entry_date,
                    'exit_date': idx,
                    'gross_return': gross_return,
                    'net_return': net_return,
                    'holding_period': len(results.loc[entry_date:idx])
                })
                entry_price = None

        return results

    def calculate_metrics(self):
        if not self.trade_returns:
            return "No trades executed"

        results = self.results
        years = (results.index[-1] - results.index[0]).days / 365.25
        risk_free_rate = 0.03

        # Strategy returns and metrics
        total_return = (results['Strategy_Equity'].iloc[-1] - self.initial_capital) / self.initial_capital
        total_return_net = (results['Strategy_Net_Equity'].iloc[-1] - self.initial_capital) / self.initial_capital
        annual_return = (1 + total_return) ** (1/years) - 1
        annual_return_net = (1 + total_return_net) ** (1/years) - 1

        daily_returns = results['Strategy'].dropna()
        daily_returns_net = results['Strategy_Net'].dropna()
        volatility = daily_returns.std() * np.sqrt(252)
        volatility_net = daily_returns_net.std() * np.sqrt(252)
        sharpe = (annual_return - risk_free_rate) / volatility if volatility != 0 else 0
        sharpe_net = (annual_return_net - risk_free_rate) / volatility_net if volatility_net != 0 else 0

        # Buy & Hold metrics
        if self.include_buy_hold:
            buy_hold_return = (results['Buy_Hold_Equity'].iloc[-1] - self.initial_capital) / self.initial_capital
            buy_hold_annual = (1 + buy_hold_return) ** (1/years) - 1
            daily_bh_returns = results['Buy_Hold'].dropna()
            bh_volatility = daily_bh_returns.std() * np.sqrt(252)
            bh_sharpe = (buy_hold_annual - risk_free_rate) / bh_volatility if bh_volatility != 0 else 0

        # Trading stats
        total_trades = len(self.trade_returns)
        winning_trades = sum(1 for t in self.trade_returns if t['net_return'] > 0)

        print("\nStrategy Performance Metrics:")
        print(f"Total Return (Gross): {total_return:.2%}")
        print(f"Total Return (Net): {total_return_net:.2%}")
        if self.include_buy_hold:
            print(f"Buy & Hold Return: {buy_hold_return:.2%}")
        print(f"Annual Return (Gross): {annual_return:.2%}")
        print(f"Annual Return (Net): {annual_return_net:.2%}")
        if self.include_buy_hold:
            print(f"Buy & Hold Annual: {buy_hold_annual:.2%}")
        print(f"Sharpe Ratio (Gross): {sharpe:.2f}")
        print(f"Sharpe Ratio (Net): {sharpe_net:.2f}")
        if self.include_buy_hold:
            print(f"Buy & Hold Sharpe: {bh_sharpe:.2f}")
        print(f"Volatility (Gross): {volatility:.2%}")
        print(f"Volatility (Net): {volatility_net:.2%}")
        if self.include_buy_hold:
            print(f"Buy & Hold Volatility: {bh_volatility:.2%}")
        print(f"Max Drawdown: {(results['Strategy_Net_Equity']/results['Strategy_Net_Equity'].cummax()-1).min():.2%}")

        print("\nTrade Statistics:")
        print(f"Total Trades: {total_trades}")
        print(f"Win Rate: {winning_trades/total_trades:.2%}")
        print(f"Average Trade Return (Gross): {np.mean([t['gross_return'] for t in self.trade_returns]):.2%}")
        print(f"Average Trade Return (Net): {np.mean([t['net_return'] for t in self.trade_returns]):.2%}")
        print(f"Best Trade (Net): {max([t['net_return'] for t in self.trade_returns]):.2%}")
        print(f"Worst Trade (Net): {min([t['net_return'] for t in self.trade_returns]):.2%}")
        print(f"Average Duration: {np.mean([t['holding_period'] for t in self.trade_returns]):.1f} periods")

    def plot_results(self):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Plot 1: Equity Curves
        self.results['Strategy_Net_Equity'].plot(ax=ax1, label='Strategy (Net)', logy=True)
        if self.include_buy_hold:
            self.results['Buy_Hold_Equity'].plot(ax=ax1, label='Buy & Hold', logy=True)
            
        ax1.set_title('Portfolio Value Over Time (Log Scale)')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Portfolio Value ($)')
        ax1.grid(True, which="both", ls="-", alpha=0.2)
        ax1.grid(True, which="minor", ls=":", alpha=0.1)
        ax1.legend()

        # Plot 2: Returns Distribution
        if self.trade_returns:
            returns = [t['net_return'] for t in self.trade_returns]
            ax2.hist(returns, bins=50, alpha=0.75, color='b', density=True)
            mean_return = np.mean(returns)
            ax2.axvline(mean_return, color='r', linestyle='--', 
                       label=f'Mean: {mean_return:.2%}')
            ax2.set_title('Net Returns Distribution (Per Trade)')
            ax2.set_xlabel('Return')
            ax2.set_ylabel('Density')
            ax2.legend()

        plt.tight_layout()
        return fig

In [134]:
class StrategyOptimizer:
    def __init__(self, 
                 symbol: str,
                 interval: str = '1h',
                 end_date: Optional[str] = None,
                 days: int = 729,
                 initial_capital: float = 10000,
                 n_jobs: int = -1,
                 data: pd.DataFrame = None):
        """
        Initialize the Strategy Optimizer
        
        Parameters:
        -----------
        symbol : str
            Trading symbol (e.g., 'BTC-USD')
        interval : str
            Trading interval (default: '1h')
        end_date : Optional[str]
            End date for optimization
        days : int
            Number of days to optimize over
        initial_capital : float
            Initial capital for backtesting
        n_jobs : int
            Number of CPU cores to use (-1 for all cores)
        data : pd.DataFrame, optional
            Pre-downloaded price data. If None, data will be downloaded.
        """
        self.symbol = symbol
        self.interval = interval
        self.end_date = end_date
        self.days = days
        self.initial_capital = initial_capital
        self.n_jobs = n_jobs
        self.best_metrics = None
        self.best_params = None
        self.best_score = float('-inf')
        
        # Download or use provided data
        if data is not None:
            self.data = data
        else:
            end_date = pd.to_datetime(end_date) if end_date else datetime.now()
            start_date = end_date - timedelta(days=days)
            print(f"Downloading data for {symbol}...")
            self.data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        
        # Define parameter bounds with descriptions
        self.param_bounds = {
            'rsi_period': {'bounds': (5, 40), 'type': int, 'description': 'RSI calculation period'},
            'rsi_smooth_period': {'bounds': (5, 30), 'type': int, 'description': 'RSI smoothing period'},
            'lower_smooth': {'bounds': (30, 60), 'type': int, 'description': 'Lower RSI threshold'},
            'upper_smooth': {'bounds': (60, 100), 'type': int, 'description': 'Upper RSI threshold'},
            'alma_period': {'bounds': (5, 50), 'type': int, 'description': 'ALMA calculation period'},
            'alma_offset': {'bounds': (0.5, 0.95), 'type': float, 'description': 'ALMA offset'},
            'alma_sigma': {'bounds': (2, 10), 'type': float, 'description': 'ALMA sigma'},
            'lz_width': {'bounds': (20, 50), 'type': int, 'description': 'Liquidity zone width'},
            'min_zone_height_pct': {'bounds': (1.0, 5.0), 'type': float, 'description': 'Minimum zone height %'},
            'entry_height_pct': {'bounds': (0, 1.0), 'type': float, 'description': 'Entry height %'},
            'exit_threshold_pct': {'bounds': (0, 3.0), 'type': float, 'description': 'Exit threshold %'},
            'atr_period': {'bounds': (5, 50), 'type': int, 'description': 'ATR period'},
            'atr_factor': {'bounds': (1, 5), 'type': float, 'description': 'ATR multiplier'},
            'pivot_period': {'bounds': (2, 10), 'type': int, 'description': 'Pivot calculation period'}
        }

    def _create_param_dict(self, params: np.ndarray) -> Dict:
        """Convert optimization parameters to strategy parameters"""
        param_dict = {}
        for i, (param_name, param_info) in enumerate(self.param_bounds.items()):
            value = params[i]
            if param_info['type'] == int:
                value = int(round(value))
            param_dict[param_name] = value
        return param_dict

    def _objective_function(self, params: np.ndarray) -> float:
        """Optimization using net metrics"""
        if hasattr(self, 'evaluation_pbar'):
            self.evaluation_pbar.update(1)

        param_dict = self._create_param_dict(params)

        try:
            strategy = TradingStrategy(
                data=self.data.copy(),
                **param_dict
            )

            backtester = Backtester(strategy, self.initial_capital, trading_cost_pct=0.004)
            metrics = backtester.calculate_metrics()

            if isinstance(metrics, str):  # No trades executed
                return -float('inf')

            # Get net metrics
            daily_returns = backtester.results['Strategy_Net'].dropna()
            volatility = daily_returns.std() * np.sqrt(252)

            total_return_net = (backtester.results['Strategy_Net_Equity'].iloc[-1] - self.initial_capital) / self.initial_capital
            years = (backtester.results.index[-1] - backtester.results.index[0]).days / 365.25
            annual_return_net = (1 + total_return_net) ** (1/years) - 1

            risk_free_rate = 0.03
            sharpe_net = (annual_return_net - risk_free_rate) / volatility if volatility != 0 else -float('inf')

            n_trades = len(backtester.trade_returns)
            win_rate = sum(1 for t in backtester.trade_returns if t['net_return'] > 0) / n_trades if n_trades > 0 else 0
            max_drawdown = (backtester.results['Strategy_Net_Equity']/backtester.results['Strategy_Net_Equity'].cummax()-1).min()

            # Hard constraints
            if n_trades < 10:
                return -float('inf')

            if win_rate < 0.40:
                return -float('inf')

            # Score using net metrics
            score = (
                sharpe_net * 0.4 +      # 40% weight on net Sharpe ratio
                annual_return_net * 0.3 + # 30% weight on net annual return
                win_rate * 0.3          # 30% weight on win rate
            )

            # Store best results if score improves
            if score > self.best_score:
                self.best_score = score
                self.best_params = param_dict
                self.best_metrics = {
                    'Net Sharpe': f"{sharpe_net:.2f}",
                    'Net Annual Return': f"{annual_return_net:.2%}",
                    'Win Rate': f"{win_rate:.2%}",
                    'Max Drawdown': f"{max_drawdown:.2%}",
                    'Total Trades': str(n_trades)
                }
                if hasattr(self, 'evaluation_pbar'):
                    self.evaluation_pbar.set_postfix({
                        'Best Score': f'{score:.2f}',
                        'Net Sharpe': f'{sharpe_net:.2f}',
                        'Net Return': f'{annual_return_net:.2%}'
                    })

            return score

        except Exception as e:
            if hasattr(self, 'evaluation_pbar'):
                self.evaluation_pbar.write(f"Error: {str(e)}")
            return -float('inf')

    def callback_function(self, xk, convergence):
        """Callback function for optimization progress tracking"""
        if hasattr(self, 'generation_pbar'):
            self.generation_pbar.update(1)
            self.generation_pbar.set_postfix({'Best Score': f'{self.best_score:.2f}'})

    def optimize(self, 
                population_size: int = 5,
                max_generations: int = 10,
                mutation: float = 0.8,
                recombination: float = 0.7) -> Tuple[Dict, Dict]:
        """Run optimization and return best parameters"""
        bounds = [info['bounds'] for info in self.param_bounds.values()]
        total_evals = population_size * max_generations

        print(f"\nOptimization Configuration:")
        print(f"Population size: {population_size}")
        print(f"Maximum generations: {max_generations}")
        print(f"Total evaluations: {total_evals}")

        with tqdm(total=total_evals, desc="Optimization Progress", unit='eval') as self.evaluation_pbar:
            result = differential_evolution(
                self._objective_function,
                bounds,
                strategy='best1bin',
                maxiter=max_generations-1,
                popsize=population_size,
                mutation=mutation,
                recombination=recombination,
                workers=1,
                updating='immediate',
                polish=False,
                callback=self.callback_function
            )

        # Print optimization summary
        self._print_optimization_summary()
        
        # Return the stored best parameters and metrics
        return self.best_params, self.best_metrics

    def _print_optimization_summary(self) -> None:
        """Print detailed optimization summary"""
        if not self.best_params or not self.best_metrics:
            print("\nNo valid optimization results found.")
            return

        print("\nOptimization Results:")
        print("====================")
        
        print("\nBest Parameters Found:")
        for param, value in self.best_params.items():
            desc = self.param_bounds[param]['description']
            print(f"{param}: {value} - {desc}")
            
        print("\nBest Performance Metrics:")
        key_metrics = ['Sharpe Ratio', 'Win Rate', 'Total Trades', 'Max Drawdown', 
                      'Annual Return', 'Volatility']
        for metric in key_metrics:
            if metric in self.best_metrics:
                print(f"{metric}: {self.best_metrics[metric]}")

    def get_best_results(self) -> Tuple[Dict, Dict]:
        """Return the best parameters and metrics found during optimization"""
        if not self.best_params or not self.best_metrics:
            raise ValueError("No optimization results available. Run optimize() first.")
        return self.best_params, self.best_metrics

In [ ]:
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=729)
data = yf.download('BTC-USD', start=start_date, end=end_date, interval='1h')

# Initialize optimizer with downloaded data
optimizer = StrategyOptimizer(
    symbol='BTC-USD',
    interval='1h',
    end_date='2024-11-14',
    days=729,
    initial_capital=10000,
    data=data
)

# Run optimization
best_params, best_metrics = optimizer.optimize(
    population_size=10,
    max_generations=30,
    mutation=0.8,
    recombination=0.7
)

[*********************100%%**********************]  1 of 1 completed


Optimization Configuration:
Population size: 10
Maximum generations: 30
Total evaluations: 300


Optimization Progress:   0%|          | 0/300 [00:00<?, ?eval/s]


Strategy Performance Metrics:
Total Return (Gross): 24.42%
Total Return (Net): 19.97%
Buy & Hold Return: 456.03%
Annual Return (Gross): 11.59%
Annual Return (Net): 9.57%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): 4.35
Sharpe Ratio (Net): 3.28
Buy & Hold Sharpe: 16.99
Volatility (Gross): 1.97%
Volatility (Net): 2.00%
Buy & Hold Volatility: 7.86%
Max Drawdown: -19.23%

Trade Statistics:
Total Trades: 4
Win Rate: 25.00%
Average Trade Return (Gross): -0.01%
Average Trade Return (Net): -0.81%
Best Trade (Net): 11.07%
Worst Trade (Net): -6.71%
Average Duration: 209.2 periods

Strategy Performance Metrics:
Total Return (Gross): 7.49%
Total Return (Net): -18.64%
Buy & Hold Return: 456.03%
Annual Return (Gross): 3.69%
Annual Return (Net): -9.83%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): 0.19
Sharpe Ratio (Net): -3.44
Buy & Hold Sharpe: 16.99
Volatility (Gross): 3.63%
Volatility (Net): 3.73%
Buy & Hold Volatility: 7.86%
Max Drawdown: -39.97%

Trade Statistics:
Total Trades: 34
Win 


Strategy Performance Metrics:
Total Return (Gross): 43.27%
Total Return (Net): 29.50%
Buy & Hold Return: 456.03%
Annual Return (Gross): 19.77%
Annual Return (Net): 13.85%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): 4.11
Sharpe Ratio (Net): 2.64
Buy & Hold Sharpe: 16.99
Volatility (Gross): 4.08%
Volatility (Net): 4.11%
Buy & Hold Volatility: 7.86%
Max Drawdown: -33.74%

Trade Statistics:
Total Trades: 12
Win Rate: 33.33%
Average Trade Return (Gross): 1.70%
Average Trade Return (Net): 0.89%
Best Trade (Net): 34.70%
Worst Trade (Net): -7.01%
Average Duration: 382.0 periods

Strategy Performance Metrics:
Total Return (Gross): 92.18%
Total Return (Net): 55.17%
Buy & Hold Return: 456.03%
Annual Return (Gross): 38.78%
Annual Return (Net): 24.66%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): 7.16
Sharpe Ratio (Net): 4.29
Buy & Hold Sharpe: 16.99
Volatility (Gross): 5.00%
Volatility (Net): 5.05%
Buy & Hold Volatility: 7.86%
Max Drawdown: -33.02%

Trade Statistics:
Total Trades: 26
Win 

...


Strategy Performance Metrics:
Total Return (Gross): -8.84%
Total Return (Net): -10.30%
Buy & Hold Return: 456.03%
Annual Return (Gross): -4.54%
Annual Return (Net): -5.31%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -19.45
Sharpe Ratio (Net): -18.80
Buy & Hold Sharpe: 16.99
Volatility (Gross): 0.39%
Volatility (Net): 0.44%
Buy & Hold Volatility: 7.86%
Max Drawdown: -10.30%

Trade Statistics:
Total Trades: 2
Win Rate: 0.00%
Average Trade Return (Gross): -4.52%
Average Trade Return (Net): -5.30%
Best Trade (Net): -4.55%
Worst Trade (Net): -6.05%
Average Duration: 10.5 periods

Strategy Performance Metrics:
Total Return (Gross): -0.33%
Total Return (Net): -1.92%
Buy & Hold Return: 456.03%
Annual Return (Gross): -0.17%
Annual Return (Net): -0.97%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -5.64
Sharpe Ratio (Net): -6.81
Buy & Hold Sharpe: 16.99
Volatility (Gross): 0.56%
Volatility (Net): 0.58%
Buy & Hold Volatility: 7.86%
Max Drawdown: -3.44%

Trade Statistics:
Total Trades: 2



Strategy Performance Metrics:
Total Return (Gross): -7.52%
Total Return (Net): -11.89%
Buy & Hold Return: 456.03%
Annual Return (Gross): -3.85%
Annual Return (Net): -6.15%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -7.74
Sharpe Ratio (Net): -9.75
Buy & Hold Sharpe: 16.99
Volatility (Gross): 0.88%
Volatility (Net): 0.94%
Buy & Hold Volatility: 7.86%
Max Drawdown: -14.78%

Trade Statistics:
Total Trades: 6
Win Rate: 0.00%
Average Trade Return (Gross): -1.28%
Average Trade Return (Net): -2.08%
Best Trade (Net): -0.19%
Worst Trade (Net): -5.10%
Average Duration: 31.3 periods

Strategy Performance Metrics:
Total Return (Gross): -15.09%
Total Return (Net): -20.73%
Buy & Hold Return: 456.03%
Annual Return (Gross): -7.88%
Annual Return (Net): -11.00%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -7.64
Sharpe Ratio (Net): -9.40
Buy & Hold Sharpe: 16.99
Volatility (Gross): 1.42%
Volatility (Net): 1.49%
Buy & Hold Volatility: 7.86%
Max Drawdown: -26.20%

Trade Statistics:
Total Trades: 


Strategy Performance Metrics:
Total Return (Gross): -6.40%
Total Return (Net): -11.19%
Buy & Hold Return: 456.03%
Annual Return (Gross): -3.26%
Annual Return (Net): -5.78%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -3.04
Sharpe Ratio (Net): -4.18
Buy & Hold Sharpe: 16.99
Volatility (Gross): 2.06%
Volatility (Net): 2.10%
Buy & Hold Volatility: 7.86%
Max Drawdown: -20.26%

Trade Statistics:
Total Trades: 6
Win Rate: 16.67%
Average Trade Return (Gross): -1.60%
Average Trade Return (Net): -2.40%
Best Trade (Net): 8.75%
Worst Trade (Net): -6.03%
Average Duration: 147.7 periods

Strategy Performance Metrics:
Total Return (Gross): -4.32%
Total Return (Net): -5.09%
Buy & Hold Return: 456.03%
Annual Return (Gross): -2.19%
Annual Return (Net): -2.59%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -13.96
Sharpe Ratio (Net): -13.64
Buy & Hold Sharpe: 16.99
Volatility (Gross): 0.37%
Volatility (Net): 0.41%
Buy & Hold Volatility: 7.86%
Max Drawdown: -5.66%

Trade Statistics:
Total Trades: 1


Strategy Performance Metrics:
Total Return (Gross): -2.36%
Total Return (Net): -3.15%
Buy & Hold Return: 456.03%
Annual Return (Gross): -1.19%
Annual Return (Net): -1.60%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -5.21
Sharpe Ratio (Net): -5.49
Buy & Hold Sharpe: 16.99
Volatility (Gross): 0.80%
Volatility (Net): 0.84%
Buy & Hold Volatility: 7.86%
Max Drawdown: -6.79%

Trade Statistics:
Total Trades: 1
Win Rate: 0.00%
Average Trade Return (Gross): -2.36%
Average Trade Return (Net): -3.15%
Best Trade (Net): -3.15%
Worst Trade (Net): -3.15%
Average Duration: 66.0 periods

Strategy Performance Metrics:
Total Return (Gross): -3.52%
Total Return (Net): -4.30%
Buy & Hold Return: 456.03%
Annual Return (Gross): -1.78%
Annual Return (Net): -2.18%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -13.35
Sharpe Ratio (Net): -13.04
Buy & Hold Sharpe: 16.99
Volatility (Gross): 0.36%
Volatility (Net): 0.40%
Buy & Hold Volatility: 7.86%
Max Drawdown: -5.66%

Trade Statistics:
Total Trades: 1
Wi


Strategy Performance Metrics:
Total Return (Gross): -3.52%
Total Return (Net): -4.30%
Buy & Hold Return: 456.03%
Annual Return (Gross): -1.78%
Annual Return (Net): -2.18%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -13.35
Sharpe Ratio (Net): -13.04
Buy & Hold Sharpe: 16.99
Volatility (Gross): 0.36%
Volatility (Net): 0.40%
Buy & Hold Volatility: 7.86%
Max Drawdown: -5.66%

Trade Statistics:
Total Trades: 1
Win Rate: 0.00%
Average Trade Return (Gross): -3.52%
Average Trade Return (Net): -4.31%
Best Trade (Net): -4.31%
Worst Trade (Net): -4.31%
Average Duration: 20.0 periods

Strategy Performance Metrics:
Total Return (Gross): -2.56%
Total Return (Net): -5.64%
Buy & Hold Return: 456.03%
Annual Return (Gross): -1.29%
Annual Return (Net): -2.87%
Buy & Hold Annual: 136.50%
Sharpe Ratio (Gross): -25.34
Sharpe Ratio (Net): -21.74
Buy & Hold Sharpe: 16.99
Volatility (Gross): 0.17%
Volatility (Net): 0.27%
Buy & Hold Volatility: 7.86%
Max Drawdown: -5.64%

Trade Statistics:
Total Trades: 4


In [ ]:
# Print results
print("\nBest Parameters:", best_params)
print("\nBest Metrics:", best_metrics)

In [ ]:
# Use best parameters with stored data
strategy = TradingStrategy(data=data, **best_params)
backtester = Backtester(strategy, initial_capital=10000, trading_cost_pct=0.004, include_buy_hold=True)
backtester.calculate_metrics()

# Plot results
strategy.plot_strategy(days_to_plot=7)
plt.show()
backtester.plot_results()
plt.show()